In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from fastai import *
from fastai.tabular import *
from fastai.callbacks import *
from fastai.text import *
from fastai.data_block import *
from fastai.metrics import *
from sklearn.model_selection import StratifiedKFold
import pickle
from fastprogress import master_bar, progress_bar
from IPython.display import FileLink
from sklearn.decomposition import PCA
import time
from swallows.training import *

In [3]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)

In [4]:
model_name = 'tab-v4-tuning-3-stage-4-hyper'

In [5]:
path=Path('data')

### Predictions

In [6]:
df_test = pd.read_parquet(path/'processed/public_test_features_stage_4.parquet')

In [7]:
df_test_ids = pd.read_csv(path/'public_test_features_IDs.csv', low_memory=False)

In [12]:
exclude_from_cats = [
    'ID', 
    'assigned_date',
    'create_date',
    'details',
    'short_description',
    'amazon_domains',
    'root_cause',
    'create_to_assigned_lag'
    'create_to_assigned_lag_log',
    'assigned_date_Elapsed'
    'create_date_Elapsed',
]

cat_names = [c for c in df_test.columns if c not in exclude_from_cats]
cont_names = [] #['create_to_assigned_lag_log']
dep_var = 'root_cause'
procs = [FillMissing, Categorify, Normalize]

In [13]:
test_data = TabularList.from_df(df_test[cat_names + cont_names], path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)

In [25]:
model_export_pickle = f'{model_name}-30-{1e-3}-{0.20}-fitted-export.pkl'
learn_pred = load_learner(path, model_export_pickle, test=test_data)
print(model_export_pickle)

tab-v4-tuning-3-stage-4-hyper-30-0.001-0.2-fitted-export.pkl


In [15]:
preds,y = learn_pred.get_preds(ds_type=DatasetType.Test)

In [19]:
pred_classes = [learn_pred.data.classes[cls] for cls in np.argmax(preds.numpy(), axis=1)]

In [22]:
df_out = pd.DataFrame({'ID': df_test['ID'], 'root_cause': pred_classes})

In [23]:
t = time.strftime('%Y_%m_%d_%H_%M')
out_fname = path/f'mle_ops_tech_{t}_{model_name}.csv'
df_out.to_csv(out_fname, index=False)
FileLink(out_fname)

/home/ec2-user/2019-ops-it-challenge/data/mle_ops_tech_2019_04_28_18_30_tab-v4-tuning-3-stage-4-hyper.csv

* val_score: 0.586620
* public leaderboard: 0.57175667112